In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/bryanleavelle/Bootcamp/Immersion/ch 4/DD-MM-2020 Instacart Basket Analysis'

In [3]:
df = pd.read_csv(os.path.join(path, 'Data', 'Original Data', '4', 'customers.csv'))

In [4]:
df_main = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data','prods_ords_with_flags.pkl' ))

In [5]:
df.shape

(206209, 10)

# WRANGLE THAT DATA

STEP 1: start with some basic description, shape, head, etc.

In [7]:
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
df.shape

(206209, 10)

In [9]:
df.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


# STEP 2: CHANGE COLUMN NAMES <br>

Only thing weird so far is the column names. SO, I will:<br>
1. change First Name to first_name
2. change Surname to last_name
3. change STATE to state
4. change Age to age
5. Gender to gender


In [10]:
# change col names as described above

df.rename(columns = {'First Name':'first_name', 'Surnam':'last_name', 'STATE':'state', 'Age':'age', 'Gender': 'gender' }, inplace = True)

In [11]:
#test for success

for col in df:
    print (col)

user_id
first_name
last_name
gender
state
age
date_joined
n_dependants
fam_status
income


NICE. <br><br>

# STEP 3: check for missing values, duplicates, mixed type variables.

In [19]:
#looking for missing values

df.isnull().sum()

user_id         0
first_name      0
last_name       0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

for some reason there were a bunch of missing first names UNTIL I FIXED the fact that it was mixed data a little further down.

In [13]:
# checking for mixed type variables

for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

first_name


looks like first_name has mixed data types. Gotta change it!

In [14]:
# change first name column to string

df['first_name'] = df['first_name'].astype('str')

In [15]:
# run again... this time with an indicator for each column showing they are not mixed.

for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
        print (col)
  else:
        print(col + ":  "+ 'all good!')

user_id:  all good!
first_name:  all good!
last_name:  all good!
gender:  all good!
state:  all good!
age:  all good!
date_joined:  all good!
n_dependants:  all good!
fam_status:  all good!
income:  all good!


checks good! now to check for duplicates.

In [16]:
# checking for duplicates

df_dups = df[df.duplicated()]

In [17]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income


no duplicate rows. Yay.

In [18]:
# just a quick safety check to make sure all user id's are unique

n = len(pd.unique(df['user_id']))
print(n)

206209


this number matches total amount of entries. Looks good!

# STEP 4: COMBINE CUST DATA SET TO MAIN DATA SET
<br>
first a quick look at both data frames:

In [20]:
# customer df head

df.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [21]:
#customer df dtypes

df.dtypes

user_id          int64
first_name      object
last_name       object
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [22]:
#main data set head

df_main.head(5)

,order_id,user_id,order_number,order_day_of_week,hour_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order,spender_category,spender_flag_loc,customer_order_frequency
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,both,Mid-range product,Regularly busy days,Average orders,10,New customer,6.367797,Low spender,Low spender,Brand-new customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Mid-range product,Least busy days,Average orders,10,New customer,6.367797,Low spender,Low spender,Regular customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Mid-range product,Least busy days,Most orders,10,New customer,6.367797,Low spender,Low spender,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Mid-range product,Least busy days,Average orders,10,New customer,6.367797,Low spender,Low spender,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Mid-range product,Least busy days,Most orders,10,New customer,6.367797,Low spender,Low spender,Non-frequent customer


In [23]:
#main data set dtypes

df_main.dtypes

order_id                       int64
user_id                        int64
order_number                   int64
order_day_of_week              int64
hour_of_order                  int64
days_since_prior_order       float64
product_id                     int64
add_to_cart_order              int64
reordered                      int64
product_name                  object
aisle_id                       int64
department_id                  int64
prices                       float64
_merge                      category
price_range_loc               object
busiest_days                  object
busiest_period_of_day         object
max_order                      int64
loyalty_flag                  object
avg_order                    float64
spender_category              object
spender_flag_loc              object
customer_order_frequency      object
dtype: object

Ok, lets attempt to join these together on User_id. Data types check out, here goes nothin.

In [25]:
del df_main['_merge']

In [26]:
# merge df and df_main

df_merged = df_main.merge(df, on = 'user_id', indicator = True)

In [27]:
#checking out my new giant beautiful df

df_merged.head(5)

,order_id,user_id,order_number,order_day_of_week,hour_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [28]:
# checking merges... 

df_merged['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [29]:
#checking total rows in df_main

df_main.shape

(32404859, 22)

Holy guacamole, it worked!

In [30]:
# new list of columnns

for col in df_merged:
    print(col)

order_id
user_id
order_number
order_day_of_week
hour_of_order
days_since_prior_order
product_id
add_to_cart_order
reordered
product_name
aisle_id
department_id
prices
price_range_loc
busiest_days
busiest_period_of_day
max_order
loyalty_flag
avg_order
spender_category
spender_flag_loc
customer_order_frequency
first_name
last_name
gender
state
age
date_joined
n_dependants
fam_status
income
_merge


In [31]:
# export new giant beautiful data set

df_merged.to_pickle(os.path.join(path, 'Data', 'Prepared Data','big_and_beautiful.pkl' ))